In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from math import floor, ceil, sqrt
from sklearn.model_selection import train_test_split
import time as time

totaltime1 = time.time()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
from os.path import exists as DoesPathExist
train_logs_path = "/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv"
train_scores_path = "/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv"
test_logs_path = "/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv"
sample_submission_path = "/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv"

if DoesPathExist(train_logs_path):
    train_logs: pd.DataFrame = pd.read_csv(train_logs_path)
else:
    train_logs: pd.DataFrame = pd.read_csv("./data/train_logs.csv")

if DoesPathExist(train_scores_path):
    train_scores = pd.read_csv(train_scores_path)
else:
    train_scores = pd.read_csv("./data/train_scores.csv")

if DoesPathExist(test_logs_path):
    test_logs = pd.read_csv(test_logs_path)
else:
    test_logs = pd.read_csv("./data/test_logs.csv")
    
if DoesPathExist(sample_submission_path):
    sample_submission = pd.read_csv(sample_submission_path)
else:
    sample_submission = pd.read_csv("./data/sample_submission.csv")
    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('expand_frame_repr', False)

## Data Preparation

In [ ]:
PUNCUATION = ['!', '"', "'", '(', ')', ',', '.', ':', ';','?', '¡', '¿', '\\']
MAX_EVENT_START_TIME = 1_800_000 # miliseconds in 30 minutes.
NUMBER_OF_INTERVALS = 1_800 # = 1_800_000 msec / 250 milisecond intervals

NUM_FEATURES = 14
NUM_FEATURES_2 = 54
FINAL_LAYER_COUNT = 12

EVENT_MAX = 6000
WORD_MAX = 800
PASTE_COUNT_DIVISOR = 10
REPLACE_COUNT_DIVISOR = 10

def ScoreOneHot(score: float) -> np.array:
    result = np.zeros(12)
    result[int(score / 0.5) - 1] = 1
    return result

def TrainingPrep(df: pd.DataFrame):

    indexs_to_drop = df[df["down_time"] > df.down_time.quantile(0.999)].index
    df.drop(indexs_to_drop, inplace=True)
    indexs_to_drop = df[df["up_time"] > df.down_time.quantile(0.999)].index
    df.drop(indexs_to_drop, inplace=True)

    return df

def DfToX(df: pd.DataFrame, minPositiveExamples: int = 0) -> np.array:
    indexs_to_drop = df[df["action_time"] > 250].index
    df = df.drop(indexs_to_drop, axis=0)
    NumUniqueIds = len(np.unique(df["id"]))

    x_train_1 = np.zeros((NumUniqueIds, NUM_FEATURES, int(NUMBER_OF_INTERVALS)))
    x_train_2 = np.zeros((NumUniqueIds, NUM_FEATURES_2))
    
    groups = df.groupby("id")

    for index, id in enumerate(np.unique(df["id"])):
        essayGroup = groups.get_group(id)
        max = essayGroup["down_time"].quantile(0.99)
        min = essayGroup["down_time"].quantile(0.01)
        dt_range = max - min
        if dt_range == 0:
            dt_range = 1
        if dt_range is None:
            dt_range = 1

        MAX_CURSOR_POSITION = essayGroup["cursor_position"].max()

        if MAX_CURSOR_POSITION is None:
            print("MAX_CURSOR_POSITION is None")

        ValueActivityCounts = essayGroup.activity.value_counts().reset_index()
        ValueEventCounts = essayGroup.down_event.value_counts().reset_index()
        ValueTextChangeCounts = essayGroup.text_change.value_counts().reset_index()

        SpaceStats = essayGroup[["id", "down_event", "event_id"]]
        SpaceStats = SpaceStats[SpaceStats["down_event"].isin(["Space"])]
        SpaceStats["word_size"] = SpaceStats["event_id"].diff()
        SpaceStats = SpaceStats.dropna(axis=0).reset_index()

        x_train_2[index][0] = essayGroup.word_count.max() / WORD_MAX
        x_train_2[index][1] = essayGroup.event_id.max() / EVENT_MAX
        x_train_2[index][2] = essayGroup.down_time.min() / 60_000

        x_train_2[index][3] = essayGroup.activity.count() / 6000
        x_train_2[index][4] = ValueActivityCounts[ValueActivityCounts["activity"] == "Input"]["count"].max() / 6000
        x_train_2[index][5] = ValueActivityCounts[ValueActivityCounts["activity"] == "Remove/Cut"]["count"].max() / 700
        x_train_2[index][6] = ValueActivityCounts[ValueActivityCounts["activity"] == "Paste"]["count"].max() / PASTE_COUNT_DIVISOR
        x_train_2[index][7] = ValueActivityCounts[ValueActivityCounts["activity"] == "Replace"]["count"].max() / REPLACE_COUNT_DIVISOR

        x_train_2[index][8] = ((x_train_2[index][7] * REPLACE_COUNT_DIVISOR) + (x_train_2[index][6] * PASTE_COUNT_DIVISOR)) / (PASTE_COUNT_DIVISOR + REPLACE_COUNT_DIVISOR)
        
        x_train_2[index][9] = ValueEventCounts[ValueEventCounts["down_event"] == "Shift"]["count"].max() / 300
        x_train_2[index][10] = ValueEventCounts[ValueEventCounts["down_event"].isin(PUNCUATION)]["count"].sum() / 100
        x_train_2[index][11] = ValueEventCounts[ValueEventCounts["down_event"].isin(["Space", "CapsLock"])]["count"].sum() / 1000
        x_train_2[index][12] = ValueEventCounts[ValueEventCounts["down_event"].isin(["Leftclick", "ArrowUp", "ArrowDown", "ArrowLeft", "ArrowRight"])]["count"].sum() / 250
        x_train_2[index][13] = ValueEventCounts[ValueEventCounts["down_event"].isin(["?", "!"])]["count"].sum() / 10

        x_train_2[index][14] = essayGroup.action_time.mean() / 100
        x_train_2[index][15] = essayGroup.action_time.sum() / 500_000
        x_train_2[index][16] = essayGroup.action_time.std() / 100
        x_train_2[index][17] = essayGroup.action_time.skew() / 20

        x_train_2[index][18] = essayGroup.down_time.mean() / 100_000
        x_train_2[index][19] = essayGroup.down_time.sum() / 100_000_000
        x_train_2[index][20] = essayGroup.down_time.std() / 500_000
        x_train_2[index][21] = essayGroup.down_time.skew() / 20
        x_train_2[index][22] = essayGroup.down_time.sem() / 10_000

        x_train_2[index][23] = essayGroup.word_count.std() / 200
        x_train_2[index][24] = essayGroup.word_count.sem() / 3
        x_train_2[index][25] = essayGroup.word_count.mean() / 350

        for i in range(10):
            if i == 0 or i == 1:
                x_train_2[index][26+i] = essayGroup[essayGroup.action_time >= (i+1)*50].action_time.count() / 2500
            elif i == 2 or i == 3:
                x_train_2[index][26+i] = essayGroup[essayGroup.action_time >= (i+1)*50].action_time.count() / 500
            else:
                x_train_2[index][26+i] = essayGroup[essayGroup.action_time >= (i+1)*50].action_time.count() / 25

        x_train_2[index][36] = ValueEventCounts[ValueEventCounts["down_event"].isin(["AudioVolumeDown", "AudioVolumeMute", "AudioVolumeUp", "MediaPlayPause", "MediaTrackNext", "MediaTrackPrevious"])]["count"].sum() / 10
        x_train_2[index][37] = ValueTextChangeCounts[ValueTextChangeCounts["text_change"].isin(["q"])]["count"].sum() / 1000
        x_train_2[index][38] = ValueTextChangeCounts[ValueTextChangeCounts["text_change"].isin([" ", "\n"])]["count"].sum() / 1000
        x_train_2[index][39] = ValueTextChangeCounts[ValueTextChangeCounts["text_change"].isin(["NoChange"])]["count"].sum() / 500
        x_train_2[index][40] = ValueTextChangeCounts[ValueTextChangeCounts["text_change"].isin(["NoChange", " ", "\n", "q"])]["count"].sum() / 500
        x_train_2[index][41] = np.sum(list(map(lambda x: x.count("=>"), essayGroup['text_change'])))

        temp_diff = essayGroup.groupby("id")["down_time"].diff(periods=1, axis=0)   
        x_train_2[index][42] = essayGroup[temp_diff > 100].dropna().groupby("id")["event_id"].count().max() / 4000
        x_train_2[index][43] = essayGroup[temp_diff > 1_000].dropna().groupby("id")["event_id"].count().max() / 200
        x_train_2[index][44] = essayGroup[temp_diff > 10_000].dropna().groupby("id")["event_id"].count().max() / 20
        x_train_2[index][45] = essayGroup[temp_diff > 100_000].dropna().groupby("id")["event_id"].count().max() / 2
        del temp_diff

        x_train_2[index][46] = SpaceStats.word_size.mean() / 10
        x_train_2[index][47] = SpaceStats.word_size.sum() / 6000
        x_train_2[index][48] = SpaceStats.word_size.skew() / 10
        x_train_2[index][49] = SpaceStats.word_size.std() / 15
        x_train_2[index][50] = SpaceStats.word_size.sem() # / 1

        x_train_2[index][51] = np.sum(list(map(lambda x: x.count("Move From"), essayGroup['activity'])))
        x_train_2[index][52] = ValueActivityCounts[ValueActivityCounts["activity"] == "Nonproduction"]["count"].max() / 500
        x_train_2[index][53] = (ValueActivityCounts[ValueActivityCounts['activity'].isin(['Input', 'Remove/Cut'])]["count"].sum() /
                               (essayGroup.up_time.max() - essayGroup.down_time.min())) * 100

        for i in range(len(x_train_2[index])):
            if np.isnan(x_train_2[index][i]):
                x_train_2[index][i] = 0

        # Build X Train 1 (convolution)
        for row in essayGroup.itertuples():

            if (row.down_time < min*0.9) or (row.down_time > max*1.1):
                continue

            start_index = floor(((row.down_time - min) / dt_range) * NUMBER_OF_INTERVALS)

            if start_index >= NUMBER_OF_INTERVALS:
                start_index = NUMBER_OF_INTERVALS - 1

            if start_index < 0:
                start_index = 0
            
            match row.activity:
                case "Nonproduction":
                    x_train_1[index][2][start_index] += 1.0
                case "Input" | "Replace":
                    # Puncuation
                    if row.down_event in PUNCUATION:
                        x_train_1[index][6][start_index] += 1.0
                    elif row.down_event == "q":
                        x_train_1[index][7][start_index] += 1.0
                    else:
                        x_train_1[index][8][start_index] += 1.0

                case "Remove/Cut":
                    x_train_1[index][4][start_index] += 1.0
                case "Paste":
                    x_train_1[index][5][start_index] += 1.0
                case _:
                    if "Move From" in row.activity:
                        x_train_1[index][3][start_index] += 1.0
            
            if x_train_1[index][0][start_index] < (row.word_count / WORD_MAX):
                x_train_1[index][0][start_index] = (row.word_count / WORD_MAX)
            
            if x_train_1[index][1][start_index] < (row.cursor_position / MAX_CURSOR_POSITION):
                x_train_1[index][1][start_index] = (row.cursor_position / MAX_CURSOR_POSITION)
            
            if x_train_1[index][9][start_index] < (row.event_id / EVENT_MAX):
                x_train_1[index][9][start_index] = (row.event_id / EVENT_MAX)

            if row.action_time < 66: # Training 0-25%
                x_train_1[index][10][start_index] += 1.0
            elif row.action_time < 93: # Training 25-50%
                x_train_1[index][11][start_index] += 1.0
            elif row.action_time < 122: # Training 50-75%
                x_train_1[index][12][start_index] += 1.0
            else: # 75%+
                x_train_1[index][13][start_index] += 1.0

        currentWordCount = 0
        for i, wordCount in enumerate(np.nditer(x_train_1[index][0])):
            if wordCount == 0:
                x_train_1[index][0] = currentWordCount
            elif currentWordCount != wordCount:
                currentWordCount = wordCount

        currentCursorPosition = 0
        for i, cursor in enumerate(np.nditer(x_train_1[index][1])):
            if cursor == 0:
                x_train_1[index][1] = currentCursorPosition
            elif cursor != currentCursorPosition:
                currentCursorPosition = cursor

        currentEventCount = 0
        for i, event in enumerate(np.nditer(x_train_1[index][1])):
            if event == 0:
                x_train_1[index][9] = currentEventCount
            elif event != currentEventCount:
                currentEventCount = event
                
    while len(x_train_1) < minPositiveExamples:
        randomExample1 = GenerateNewExample(x_train=x_train_1)
        randomExample2 = GenerateNewExample(x_train=x_train_2)
        x_train_1 = np.concatenate((x_train_1, [randomExample1]))
        x_train_2 = np.concatenate((x_train_2, [randomExample2]))

    x_train_1 = np.transpose(x_train_1, axes=(0, 2, 1))
    return x_train_1, x_train_2

def GenerateNewExample(x_train: np.array):
    example_index = np.random.choice(len(x_train), 1)[0]
    Noise = (0.005 * np.random.sample(x_train[0].shape)) - 0.0025
    result = x_train[example_index] *  (1 + Noise)
    return result

In [ ]:
TrainingPrep(train_logs)

In [ ]:
score_labels, score_counts = np.unique(train_scores.score, return_counts=True)

train_x_1 = dict(zip(score_labels, [np.zeros((floor(count - (0.2*count)), NUM_FEATURES, int(NUMBER_OF_INTERVALS))) for count in score_counts]))
test_x_1 = dict(zip(score_labels, [np.zeros((floor(0.2*count), NUM_FEATURES, int(NUMBER_OF_INTERVALS))) for count in score_counts]))

train_x_2 = dict(zip(score_labels, [np.zeros((floor(count - (0.2*count)), NUM_FEATURES, int(NUMBER_OF_INTERVALS))) for count in score_counts]))
test_x_2 = dict(zip(score_labels, [np.zeros((floor(0.2*count), NUM_FEATURES, int(NUMBER_OF_INTERVALS))) for count in score_counts]))


for score_index, score in enumerate(score_labels):
    count = score_counts[score_index]
    ids = train_scores[train_scores["score"] == (score)]
    temp_train_x_1, temp_train_x_2 = DfToX(train_logs[train_logs["id"].isin(ids.id)], minPositiveExamples=50)

    cutoff = floor(count*0.2)
    train_x_1[score] = temp_train_x_1[cutoff:]
    test_x_1[score] = temp_train_x_1[:cutoff]

    train_x_2[score] = temp_train_x_2[cutoff:]
    test_x_2[score] = temp_train_x_2[:cutoff]

In [ ]:
SHAPE_INPUT_X1 = train_x_1[0.5].shape[1:]
SHAPE_INPUT_X2 = train_x_2[0.5].shape[1:]

print(SHAPE_INPUT_X1)
print(SHAPE_INPUT_X2)

## Layers

In [ ]:
def GenerateModelLayers():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=160, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=8, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=512, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=2048, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=512, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=128, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=512, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=512, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=128, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=128, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

binary_threshold = 0.6
loss_bin = [tf.keras.losses.binary_crossentropy]
metrics_bin = [
               tf.keras.metrics.BinaryAccuracy(threshold=binary_threshold), 
               tf.keras.metrics.FalsePositives(thresholds=binary_threshold), 
               tf.keras.metrics.FalseNegatives(thresholds=binary_threshold), 
               tf.keras.metrics.TruePositives(thresholds=binary_threshold), 
               tf.keras.metrics.TrueNegatives(thresholds=binary_threshold)
            ]

Split_1_Input, Split_2_Input, Split_Output = GenerateModelLayers()
modelSplit = tf.keras.models.Model(inputs=[Split_1_Input, Split_2_Input], outputs=Split_Output, name="Split")
modelSplit.compile(optimizer='adam', loss=loss_bin, metrics=metrics_bin)
modelSplit.summary()

del modelSplit
del Split_1_Input
del Split_2_Input
del Split_Output

## Train Function

In [ ]:
def TrainScore(score: float, epochs: int = 10, num_models: int = 5, layerGenerator = GenerateModelLayers):
    assert (score in train_x_1.keys())
    assert (score in train_x_2.keys())
    x1_train_pos = np.zeros((0, SHAPE_INPUT_X1[0], SHAPE_INPUT_X1[1]))
    x2_train_pos = np.zeros((0, SHAPE_INPUT_X2[0]))
    y_train_pos = np.zeros((0))

    x1_train_neg = np.zeros((0, SHAPE_INPUT_X1[0], SHAPE_INPUT_X1[1]))
    x2_train_neg = np.zeros((0, SHAPE_INPUT_X2[0]))
    y_train_neg = np.zeros((0))

    for key in train_x_1.keys():
        if key == score:
            x1_train_pos = np.concatenate([x1_train_pos, train_x_1[key]])
            x2_train_pos = np.concatenate([x2_train_pos, train_x_2[key]])
            y_train_pos = np.concatenate([y_train_pos, np.ones(len(train_x_1[key]))])
        else:
            x1_train_neg = np.concatenate([x1_train_neg, train_x_1[key]])
            x2_train_neg = np.concatenate([x2_train_neg, train_x_2[key]])
            y_train_neg = np.concatenate([y_train_neg, np.zeros(len(train_x_1[key]))])
            
    Split_1_Input, Split_2_Input, Split_Output = layerGenerator()
    models = [tf.keras.models.Model(inputs=[Split_1_Input, Split_2_Input], outputs=Split_Output, name=f"Score_{score}_{x}") for x in range(num_models)]

    x1_train_pos, x1_valid_pos, x2_train_pos, x2_valid_pos = train_test_split(x1_train_pos, x2_train_pos, test_size=0.25, shuffle = True)
    x1_train_neg, x1_valid_neg, x2_train_neg, x2_valid_neg = train_test_split(x1_train_neg, x2_train_neg, test_size=0.25, shuffle = True)

    y_train_pos = np.ones(len(x1_train_pos))
    y_valid_pos = np.ones(len(x1_valid_pos))
    y_train_neg = np.zeros(len(x1_train_neg))
    y_valid_neg = np.zeros(len(x1_valid_neg))

    positiveAmount = len(y_train_pos)
    negativeAmount = len(y_train_neg)
    amounts = (len(y_train_pos), len(y_valid_pos), len(y_train_neg), len(y_valid_neg))

    x1_train = np.concatenate((x1_train_pos, x1_train_neg))
    x1_valid = np.concatenate((x1_valid_pos, x1_valid_neg))
    x2_train = np.concatenate((x2_train_pos, x2_train_neg))
    x2_valid = np.concatenate((x2_valid_pos, x2_valid_neg))
    y_train = np.concatenate((y_train_pos, y_train_neg))
    y_valid = np.concatenate((y_valid_pos, y_valid_neg))

    print(y_train.shape)
    print(f"Positive: {positiveAmount + len(y_valid_pos)}; Train: {positiveAmount}; Valid: {len(y_valid_pos)}")
    print(f"Negative: {negativeAmount + len(y_valid_neg)}; Train: {negativeAmount}; Valid: {len(y_valid_neg)}")
    positiveWeight = (positiveAmount + negativeAmount) / (2 * positiveAmount)
    negativeWeight = (positiveAmount + negativeAmount) / (2 * negativeAmount)

    best_loss = 10000
    best_model = None
    best_history = None
    for model in models:
        print(f"{model.name}")
        model.compile(optimizer='adam', loss=loss_bin, metrics=metrics_bin)
        history = model.fit(x=[x1_train, x2_train], y=y_train, shuffle=True, validation_data=([x1_valid, x2_valid], y_valid), epochs=epochs, class_weight={0: negativeWeight, 1: positiveWeight})
        if model.get_metrics_result()["loss"] < best_loss:
            best_loss = model.get_metrics_result()["loss"]
            best_model = model
            best_history = history
        print()
    return best_model, best_history, amounts

def DisplayHistory(history, epochs, amounts):
    epochs_range = range(epochs)
    rows = 3
    cols = 2
    displayIndex = 0

    posTrainAmount = amounts[0] if amounts[0] != 0 else 1
    posValidAmount = amounts[1] if amounts[1] != 0 else 1
    negTrainAmount = amounts[2] if amounts[2] != 0 else 1
    negValidAmount = amounts[3] if amounts[3] != 0 else 1

    displayIndex += 1
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    fig = plt.figure(figsize=(16, 14))
    #fig.suptitle("Training History")
    ax = fig.add_subplot(rows, cols, displayIndex)
    ax.set_ylim(0, 3)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    displayIndex += 1
    data = history.history['binary_accuracy']
    val_data = history.history['val_binary_accuracy']
    ax = fig.add_subplot(rows, cols, displayIndex)
    ax.set_ylim(0, 1)
    plt.plot(epochs_range, data, label='Training')
    plt.plot(epochs_range, val_data, label='Validation')
    plt.legend(loc='upper left')
    plt.title('Binary Accuracy')

    displayIndex += 1
    data = history.history['true_positives']
    val_data = history.history['val_true_positives']
    ax = fig.add_subplot(rows, cols, displayIndex)
    plt.plot(epochs_range, np.divide(data, posTrainAmount), label='Training')
    plt.plot(epochs_range, np.divide(val_data, posValidAmount), label='Validation')
    plt.legend(loc='upper left')
    plt.title('True Positives')

    displayIndex += 1
    data = history.history['false_positives']
    val_data = history.history['val_false_positives']
    ax = fig.add_subplot(rows, cols, displayIndex)
    plt.plot(epochs_range, np.divide(data, negTrainAmount), label='Training')
    plt.plot(epochs_range, np.divide(val_data, negValidAmount), label='Validation')
    plt.legend(loc='upper left')
    plt.title('False Positives')

    displayIndex += 1
    data = history.history['true_negatives']
    val_data = history.history['val_true_negatives']
    ax = fig.add_subplot(rows, cols, displayIndex)
    #ax.set_ylim(0, 3)
    plt.plot(epochs_range, np.divide(data, negTrainAmount), label='Training')
    plt.plot(epochs_range, np.divide(val_data, negValidAmount), label='Validation')
    plt.legend(loc='upper left')
    plt.title('True Negatives')

    displayIndex += 1
    data = history.history['false_negatives']
    val_data = history.history['val_false_negatives']
    ax = fig.add_subplot(rows, cols, displayIndex)
    #ax.set_ylim(0, 3)
    plt.plot(epochs_range, np.divide(data, posTrainAmount), label='Training')
    plt.plot(epochs_range, np.divide(val_data, posValidAmount), label='Validation')
    plt.legend(loc='upper left')
    plt.title('False Negatives')

## Training

### 0.5

In [ ]:
def GenerateModelLayers05():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=160, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    # layers_1 = tf.keras.layers.Conv1D(filters=40, 
    #                                 kernel_size = (4,), 
    #                                 strides=1, 
    #                                 padding="SAME", 
    #                                 activation='relu', 
    #                                 kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
    #                                 )(layers_1)
    # layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    # layers_1 = tf.keras.layers.Conv1D(filters=8, 
    #                                 kernel_size = (4,), 
    #                                 strides=1, 
    #                                 padding="SAME", 
    #                                 activation='relu', 
    #                                 kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
    #                                 )(layers_1)
    # layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=512, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=128, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 25
model05, history, amounts = TrainScore(0.5, epochs=epochs, layerGenerator=GenerateModelLayers05)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 1.0

In [ ]:
def GenerateModelLayers10():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=160, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=512, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=128, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 20
model10, history, amounts = TrainScore(1.0, epochs=epochs, layerGenerator=GenerateModelLayers10)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 1.5

In [ ]:
def GenerateModelLayers15():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=160, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=24, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=256, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=256, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=256, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=128, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=128, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 20
model15, history, amounts = TrainScore(1.5, epochs=epochs, layerGenerator=GenerateModelLayers15)

In [ ]:
print(history.history.keys())
DisplayHistory(history, epochs, amounts)

### 2.0

In [ ]:
def GenerateModelLayers20():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=160, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=512, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=128, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 25
model20, history, amounts = TrainScore(2.0, epochs=epochs, layerGenerator=GenerateModelLayers20)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 2.5

In [ ]:
def GenerateModelLayers25():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=160, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=6, 
                                    kernel_size = (2,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=512, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=256, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=256, activation='relu')(layers_2)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_2 = tf.keras.layers.Dense(units=64, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 20
model25, history, amounts = TrainScore(2.5, epochs=epochs, layerGenerator=GenerateModelLayers25)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 3.0

In [ ]:
def GenerateModelLayers30():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=160, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=8, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=512, activation='relu')(layers_1)
    # layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    # layers_1 = tf.keras.layers.Dense(units=2048, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=1024, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=512, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=256, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=128, activation='relu')(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=96, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output
epochs = 20
model30, history, amounts = TrainScore(3.0, epochs=epochs, layerGenerator= GenerateModelLayers30)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 3.5

In [ ]:
def GenerateModelLayers35():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=160, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)


    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=64, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=512, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=128, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=64, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=128, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=72, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=36, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output


epochs = 20
model35, history, amounts = TrainScore(3.5, epochs=epochs, layerGenerator=GenerateModelLayers35)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 4.0

In [ ]:
def GenerateModelLayers40():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=80, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=32, 
                                    kernel_size = (2,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=24, 
                                    kernel_size = (2,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=256, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=64, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=256, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=144, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=64, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=128, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=36, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 50
model40, history, amounts = TrainScore(4.0, epochs=epochs, layerGenerator=GenerateModelLayers40)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 4.5

In [ ]:
def GenerateModelLayers45():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=80, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=32, 
                                    kernel_size = (2,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=24, 
                                    kernel_size = (2,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=256, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=64, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=256, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=144, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=64, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=128, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=36, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 25
model45, history, amounts = TrainScore(4.5, epochs=epochs, layerGenerator=GenerateModelLayers45)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 5.0

In [ ]:
def GenerateModelLayers50():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=80, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=32, 
                                    kernel_size = (2,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=24, 
                                    kernel_size = (2,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=256, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=64, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=256, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=64, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=128, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=64, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=36, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 20
model50, history, amounts = TrainScore(5.0, epochs=epochs, num_models=3, layerGenerator=GenerateModelLayers50)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 5.5

In [ ]:
def GenerateModelLayers55():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=80, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=32, 
                                    kernel_size = (2,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=24, 
                                    kernel_size = (2,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=256, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=64, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=256, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=144, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=64, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=128, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=36, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 25
model55, history, amounts = TrainScore(5.5, epochs=epochs, layerGenerator=GenerateModelLayers55)

In [ ]:
DisplayHistory(history, epochs, amounts)

### 6.0

In [ ]:
def GenerateModelLayers60():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=160, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=40, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=8, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=512, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=128, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=24, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=128, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=12, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output]))
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=80, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)

    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(layers_3)
    return model_1.input, model_2.input, layers_Output

epochs = 40
model60, history, amounts = TrainScore(6.0, epochs=epochs, layerGenerator = GenerateModelLayers60)

In [ ]:
DisplayHistory(history, epochs, amounts)

## Model Combination

In [ ]:
AllModels = {
    0.5: model05,
    1.0: model10,
    1.5: model15,
    2.0: model20,
    2.5: model25,
    3.0: model30,
    3.5: model35,
    4.0: model40,
    4.5: model45,
    5.0: model50,
    5.5: model55,
    6.0: model60
}

### Data Preperation

In [ ]:
TOTAL_TRAIN = 0

for score_index, score in enumerate(score_labels):
    TOTAL_TRAIN += train_x_1[score].shape[0]

SHAPE_INPUT_XP = (TOTAL_TRAIN, 12)

com_train_x_1 = np.zeros(shape=(TOTAL_TRAIN, train_x_1[0.5].shape[1], train_x_1[0.5].shape[2]))
com_train_x_2 = np.zeros(shape=(TOTAL_TRAIN, train_x_2[0.5].shape[1]))
com_train_x_P = np.zeros(shape=SHAPE_INPUT_XP)
com_train_y = np.zeros(shape=(TOTAL_TRAIN))


com_index = 0
for score_index, score in enumerate(score_labels):
    print(score)
    for index, x in enumerate(train_x_1[score]):
        com_train_x_1[com_index] = train_x_1[score][index]
        com_train_x_2[com_index] = train_x_2[score][index]
        com_train_y[com_index] = score

        for score_index_2, score_2 in enumerate(score_labels):
            prediction = AllModels[score_2].predict(
                x=[np.array([train_x_1[score][index]]), np.array([train_x_2[score][index]])],
                verbose=0
                )
            com_train_x_P[com_index][score_index_2] = prediction
        
        com_index += 1

In [ ]:
print(f"Com_Train_x1: Shape - {com_train_x_1.shape}, Ex: {com_train_x_1[0]}")
print(f"Com_Train_x2: Shape - {com_train_x_2.shape}, Ex: {com_train_x_2[0]}")
print(f"Com_Train_xP: Shape - {com_train_x_P.shape}, Ex: {com_train_x_P[0]}")
print(f"Com_Train_y:  Shape - {com_train_y.shape}, Ex: {com_train_y[0]}")

### Layer Generation

In [ ]:
def GenerateModelLayersFinalModel():
    input_shape_1 = SHAPE_INPUT_X1
    input_shape_2 = SHAPE_INPUT_X2
    input_shape_3 = (12,)

    input_1 = tf.keras.Input(shape=input_shape_1, dtype=np.float64)
    input_2 = tf.keras.Input(shape=input_shape_2, dtype=np.float64)
    input_3 = tf.keras.Input(shape=input_shape_3, dtype=np.float64)

    layers_1 = tf.keras.layers.Normalization()(input_1)
    layers_1 = tf.keras.layers.Conv1D(filters=80, 
                                    kernel_size = (4,), 
                                    strides=2, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Conv1D(filters=8, 
                                    kernel_size = (4,), 
                                    strides=1, 
                                    padding="SAME", 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
                                    )(layers_1)
    layers_1 = tf.keras.layers.MaxPool1D()(layers_1)

    layers_1 = tf.keras.layers.Flatten()(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.4)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=512, activation='relu')(layers_1)
    layers_1 = tf.keras.layers.Dropout(0.2)(layers_1)
    layers_1 = tf.keras.layers.Dense(units=64, activation='relu')(layers_1)
    model_1 = tf.keras.Model(inputs = input_1, outputs=layers_1)

    layers_2 = tf.keras.layers.Normalization()(input_2)
    layers_2 = tf.keras.layers.Dense(units=NUM_FEATURES_2, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dense(units=512, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.4)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=1024, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    layers_2 = tf.keras.layers.Dense(units=64, activation='relu')(layers_2)
    layers_2 = tf.keras.layers.Dropout(0.2)(layers_2)
    model_2 = tf.keras.models.Model(inputs = input_2, outputs=layers_2)

    layers_3 = tf.keras.layers.Flatten()(input_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=256, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=128, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.4)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    layers_3 = tf.keras.layers.Dropout(0.2)(layers_3)
    layers_3 = tf.keras.layers.Dense(units=12, activation='relu')(layers_3)
    model_3 = tf.keras.models.Model(inputs = input_3, outputs=layers_3)

    layers_4 = tf.keras.layers.Dense(units=16, activation='relu')(tf.keras.layers.concatenate([model_1.output, model_2.output, model_3.output]))
    layers_4 = tf.keras.layers.Dropout(0.4)(layers_4)
    layers_4 = tf.keras.layers.Dense(units=16, activation='relu')(layers_4)
    layers_4 = tf.keras.layers.Dropout(0.4)(layers_4)
    layers_Output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.linear)(layers_4)
    return model_1.input, model_2.input, model_3.input, layers_Output

loss_final = [tf.keras.losses.MeanAbsoluteError()]
metrics_final = [tf.keras.metrics.RootMeanSquaredError()]

Split_1_Input, Split_2_Input, Split_3_Input, Split_Output = GenerateModelLayersFinalModel()
modelCom = tf.keras.models.Model(inputs=[Split_1_Input, Split_2_Input, Split_3_Input], outputs=Split_Output, name="Final")
modelCom.compile(optimizer='adam', loss=loss_final, metrics=metrics_final)
modelCom.summary()

del modelCom
del Split_1_Input
del Split_2_Input
del Split_3_Input
del Split_Output

### Training

In [ ]:
num_models = 1
epochs = 10
Final_1_Input, Final_2_Input, Final_3_Input, Final_Output = GenerateModelLayersFinalModel()
FinalModels = [tf.keras.models.Model(inputs=[Final_1_Input, Final_2_Input, Final_3_Input], outputs=Final_Output, name=f"Final_Model_{x}") for x in range(num_models)]

best_loss_final = 10000
best_model_final = None
best_history_final = None

for model in FinalModels:
    print(f"{model.name}")
    model.compile(optimizer='adam', loss=loss_final, metrics=metrics_final)
    history_final = model.fit(x=[com_train_x_1, com_train_x_2, com_train_x_P], y=com_train_y, shuffle=True, validation_split=0.25, epochs=epochs)
    if model.get_metrics_result()["loss"] < best_loss_final:
        best_loss_final = model.get_metrics_result()["loss"]
        best_model_final = model
        best_history_final = history_final
    print()

In [ ]:
epochs_range = range(epochs)
rows = 1
cols = 2

loss = best_history_final.history['loss']
val_loss = best_history_final.history['val_loss']
fig = plt.figure(figsize=(16, 8))
fig.suptitle("Best Final Training History")
ax = fig.add_subplot(rows, cols, 1)
ax.set_ylim(0, 3)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

data = best_history_final.history['root_mean_squared_error']
val_data = best_history_final.history['val_root_mean_squared_error']
ax = fig.add_subplot(rows, cols, 2)
#ax.set_ylim(0, 3)
plt.plot(epochs_range, data, label='Training')
plt.plot(epochs_range, val_data, label='Validation')
plt.legend(loc='upper left')
plt.title('RMSE')

### Testing Evalutaion

In [ ]:
TOTAL_TEST = 0

for score_index, score in enumerate(score_labels):
    TOTAL_TEST += test_x_1[score].shape[0]

# com_test_x_1 = np.zeros(shape=(TOTAL_TEST, test_x_1[0.5].shape[1], test_x_1[0.5].shape[2]))
# com_test_x_2 = np.zeros(shape=(TOTAL_TEST, test_x_2[0.5].shape[1]))
com_test_x_P = np.zeros(shape=(TOTAL_TEST, 12))
com_test_y = np.zeros(shape=(TOTAL_TEST))
com_test_pred = np.zeros(shape=(TOTAL_TEST))

com_index = 0
for score_index, score in enumerate(score_labels):
    print(score)
    for index, x in enumerate(test_x_1[score]):
        # com_test_x_1[com_index] = test_x_1[score][index]
        # com_test_x_2[com_index] = test_x_2[score][index]
        individual_predictions = np.zeros(12)
        com_test_y[com_index] = score

        for score_index_2, score_2 in enumerate(score_labels):
            prediction = AllModels[score_2].predict(x=[np.array([test_x_1[score][index]]), np.array([test_x_2[score][index]])], verbose=0)
            individual_predictions[score_index_2] = prediction
        
        pred = best_model_final.predict(x=[
            np.array([test_x_1[score][index]]), 
            np.array([test_x_2[score][index]]), 
            np.array([individual_predictions])
            ],
            verbose=0
        )

        com_test_pred[com_index] = pred
        com_index += 1

In [ ]:
#print(com_test_pred - com_test_y)
np.sqrt(np.mean((com_test_pred-com_test_y)**2))

## Generating Predictions

In [ ]:
# 2c75e15a --> 4.0
# 2c7997a3 --> 3.5
# 2d299968 --> 5.0
# test_logs = pd.read_csv("./data/personaltest.csv")

predictions = []
x_pred, x2_pred = DfToX(test_logs)
BinaryScorePredictions = np.array([
    AllModels[0.5].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[1.0].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[1.5].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[2.0].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[2.5].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[3.0].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[3.5].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[4.0].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[4.5].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[5.0].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[5.5].predict(x=[x_pred, x2_pred], verbose=0),
    AllModels[6.0].predict(x=[x_pred, x2_pred], verbose=0)
])
BinaryScorePredictions = np.transpose(BinaryScorePredictions, axes=(1, 0, 2))

print(x_pred.shape)
print(x2_pred.shape)
print(BinaryScorePredictions.shape)
print(BinaryScorePredictions)

predictions_final = best_model_final.predict(x=[x_pred, x2_pred, BinaryScorePredictions], verbose=0)
predictions = []
for index, id in enumerate(test_logs["id"].unique()):
    pred_score = 3.5
    try:
        pred_score = predictions_final[index][0]
        print(pred_score)
    except ValueError as ex:
        pred_score = 3.5
        print(f"VALUE ERROR OCCURED. ERROR: {ex}")
    except Exception as ex:
        print(f"ERROR: {ex}")
        pred_score = 3.5
    finally:
        predictions.append((id, pred_score))

ids = [p[0] for p in predictions]
scores = [p[1] for p in predictions]

sample_submission["id"] = ids
sample_submission["score"] = scores
sample_submission[["id", "score"]].to_csv("submission.csv", index=False)

## Timing

In [ ]:
totaltime2 = time.time()
print(totaltime2 - totaltime1)

In [ ]:
print((totaltime2-totaltime1)/60)